# ResearchTrader API Testing

This notebook tests each endpoint of the ResearchTrader API to ensure everything is working correctly.

In [1]:
import httpx
import asyncio
import json
from IPython.display import display, JSON
import time
from pprint import pprint

In [2]:
# API configuration
API_BASE_URL = "http://localhost:8000"

# Helper function to display JSON responses nicely
def display_json(data):
    """Display JSON data nicely formatted in the notebook"""
    if data:
        display(JSON(data))
    else:
        print("No data to display")

## 1. Testing Search Endpoint

In [3]:
async def test_search_papers(query="reinforcement learning trading", max_results=5):
    """Test the search endpoint"""
    print(f"Testing search endpoint with query: '{query}'")
    start_time = time.time()
    
    async with httpx.AsyncClient(timeout=30.0) as client:
        response = await client.get(
            f"{API_BASE_URL}/search/",
            params={"query": query, "max_results": max_results}
        )
        
        print(f"Status code: {response.status_code}")
        elapsed = time.time() - start_time
        print(f"Response time: {elapsed:.2f} seconds")
        
        if response.status_code == 200:
            response_data = response.json()
            
            # Display summary of results
            if "papers" in response_data and len(response_data["papers"]) > 0:
                papers = response_data["papers"]
                print(f"Found {len(papers)} papers")
                
                # Return the first paper ID for use in other tests
                paper_id = papers[0]["id"]
                print(f"First paper ID: {paper_id}")
                print(f"Title: {papers[0]['title']}")
                
                return paper_id, response_data
            else:
                print("No papers found or error in response structure")
                return None, response_data
        else:
            print(f"Error: {response.text}")
            return None, None

In [4]:
# Run the search test
paper_id, search_results = await test_search_papers("momentum trading strategies")

# Display full results
if search_results:
    print("\nDetailed search results:")
    # Only show the first 2 papers to keep output manageable
    search_results_limited = search_results.copy()
    if "papers" in search_results_limited and len(search_results_limited["papers"]) > 2:
        search_results_limited["papers"] = search_results_limited["papers"][:2]
    display_json(search_results_limited)

Testing search endpoint with query: 'momentum trading strategies'
Status code: 200
Response time: 2.64 seconds
Found 5 papers
First paper ID: http://arxiv.org/abs/2504.10789v1
Title: Can Large Language Models Trade? Testing Financial Theories with LLM Agents in Market Simulations

Detailed search results:


<IPython.core.display.JSON object>

## 2. Testing Paper Retrieval by ID

In [5]:
async def test_get_paper(paper_id):
    """Test getting a specific paper by ID"""
    if not paper_id:
        print("No paper ID provided")
        return None
        
    print(f"Testing get paper endpoint for ID: {paper_id}")
    start_time = time.time()
    
    async with httpx.AsyncClient(timeout=30.0) as client:
        response = await client.get(f"{API_BASE_URL}/search/{paper_id}")
        
        print(f"Status code: {response.status_code}")
        elapsed = time.time() - start_time
        print(f"Response time: {elapsed:.2f} seconds")
        
        if response.status_code == 200:
            paper = response.json()
            print(f"Paper title: {paper.get('title', 'N/A')}")
            print(f"Authors: {', '.join(paper.get('authors', []))}")
            return paper
        else:
            print(f"Error fetching paper: {response.text}")
            return None

In [6]:
# Run the get paper test
paper = await test_get_paper('2504.10789v1')

# Display full paper details
if paper:
    print("\nDetailed paper information:")
    display_json(paper)

Testing get paper endpoint for ID: 2504.10789v1
Status code: 200
Response time: 4.93 seconds
Paper title: Can Large Language Models Trade? Testing Financial Theories with LLM Agents in Market Simulations
Authors: Alejandro Lopez-Lira

Detailed paper information:


<IPython.core.display.JSON object>

## 3. Testing Paper Summarization

In [7]:
async def test_paper_summary(paper_id, force_refresh=False):
    """Test generating paper summary"""
    if not paper_id:
        print("No paper ID provided")
        return None
        
    print(f"Testing summary endpoint for paper ID: {paper_id}")
    print(f"Force refresh: {force_refresh}")
    start_time = time.time()
    
    async with httpx.AsyncClient(timeout=120.0) as client:  # Longer timeout for LLM processing
        response = await client.get(
            f"{API_BASE_URL}/summarize/{paper_id}",
            params={"force_refresh": force_refresh}
        )
        
        print(f"Status code: {response.status_code}")
        elapsed = time.time() - start_time
        print(f"Response time: {elapsed:.2f} seconds")
        
        if response.status_code == 200:
            summary = response.json()
            # Display key information
            print(f"Using full text: {summary.get('has_full_text', False)}")
            print(f"Objective: {summary.get('objective', 'N/A')[:100]}...")
            print(f"Methods: {len(summary.get('methods', []))} items")
            print(f"Results: {len(summary.get('results', []))} items")
            print(f"Trading applications: {len(summary.get('trading_applications', []))} items")
            print(f"Implementation complexity: {summary.get('implementation_complexity', 'N/A')}")
            print(f"Keywords: {', '.join(summary.get('keywords', []))}")
            return summary
        else:
            print(f"Error generating summary: {response.text}")
            return None

In [8]:
# Run the summary test
summary = await test_paper_summary('2504.10789v1')

# Display certain sections in more detail
if summary:
    print("\nMethods:")
    for method in summary.get("methods", []):
        print(f"- {method}")
        
    print("\nTrading Applications:")
    for app in summary.get("trading_applications", []):
        print(f"- {app}")
        
    print("\nData Requirements:")
    for req in summary.get("data_requirements", []):
        print(f"- {req}")

Testing summary endpoint for paper ID: 2504.10789v1
Force refresh: False
Status code: 200
Response time: 12.75 seconds
Using full text: True
Objective: To test the capabilities of large language models (LLMs) as trading agents in a simulated financial ...
Methods: 5 items
Results: 5 items
Trading applications: 5 items
Implementation complexity: Medium
Keywords: Large Language Models, Agent-Based Trading, Market Simulation, Price Discovery, Systemic Risk, Trading Strategies, Market Dynamics, AI Trading

Methods:
- Open-source simulation framework for agent-based trading
- Continuous double-auction market mechanism
- Prompt engineering for defining agent strategies
- Structured output format for agent decision-making
- Comprehensive data collection and analysis system

Trading Applications:
- Development of LLM-based trading systems that can adapt to various market conditions.
- Utilization of the framework for backtesting and validating trading strategies.
- Exploration of new trading s

## 4. Testing Paper Full Text Retrieval

In [10]:
async def test_paper_text(paper_id):
    """Test retrieving paper full text"""
    if not paper_id:
        print("No paper ID provided")
        return None
        
    print(f"Testing paper text endpoint for paper ID: {paper_id}")
    start_time = time.time()
    
    async with httpx.AsyncClient(timeout=60.0) as client:
        response = await client.get(f"{API_BASE_URL}/summarize/text/{paper_id}")
        
        print(f"Status code: {response.status_code}")
        elapsed = time.time() - start_time
        print(f"Response time: {elapsed:.2f} seconds")
        
        if response.status_code == 200:
            paper_text = response.json()
            text_length = len(paper_text.get("full_text", ""))
            print(f"Retrieved {text_length} characters of text")
            
            # Show the first 500 characters of text as preview
            print("\nText preview:")
            print(paper_text.get("full_text", "")[:500] + "...")
            
            # Show available sections if any
            sections = paper_text.get("sections", {})
            if sections:
                print(f"\nIdentified {len(sections)} sections:")
                for section, content in sections.items():
                    print(f"- {section}: {len(content)} characters")
            
            return paper_text
        else:
            print(f"Error retrieving paper text: {response.text}")
            return None

In [13]:
paper_text = await test_paper_text('2504.10789v1')

Testing paper text endpoint for paper ID: 2504.10789v1
Status code: 200
Response time: 0.85 seconds
Retrieved 76480 characters of text

Text preview:
Can Large Language Models Trade? Testing Financial

Theories with LLM Agents in Market Simulations

Alejandro Lopez-Lira∗
First Version: November 29, 2024; Current Version: April 16, 2025

Abstract

This paper presents a realistic simulated stock market where large language models

(LLMs) act as heterogeneous competing trading agents. The open-source framework

incorporates a persistent order book with market and limit orders, partial fills, div-

idends, and equilibrium clearing alongside agent...


## 5. Testing Q&A Endpoint

In [ ]:
async def test_qa(paper_id, question):
    """Test asking a question about a paper"""
    if not paper_id:
        print("No paper ID provided")
        return None
        
    print(f"Testing QA endpoint with paper ID: {paper_id}")
    print(f"Question: {question}")
    start_time = time.time()
    
    async with httpx.AsyncClient(timeout=60.0) as client:  # Longer timeout for LLM processing
        response = await client.post(
            f"{API_BASE_URL}/qa/",
            json={"question": question, "paper_ids": [paper_id]}
        )
        
        print(f"Status code: {response.status_code}")
        elapsed = time.time() - start_time
        print(f"Response time: {elapsed:.2f} seconds")
        
        if response.status_code == 200:
            qa_result = response.json()
            print(f"Confidence: {qa_result.get('confidence', 0)}")
            print(f"\nAnswer: {qa_result.get('answer', '')}")
            
            # Display suggested follow-up questions if available
            suggestions = qa_result.get('suggestions', [])
            if suggestions:
                print("\nSuggested follow-up questions:")
                for suggestion in suggestions:
                    print(f"- {suggestion}")
                    
            return qa_result
        else:
            print(f"Error getting answer: {response.text}")
            return None

In [ ]:
# Run the Q&A test
qa_result = await test_qa(
    paper_id, 
    "What are the main trading applications of this research and how difficult would it be to implement?"
)

# Display full Q&A result
if qa_result:
    print("\nFull Q&A result:")
    display_json(qa_result)

## 6. Testing Strategy Generation Endpoint

In [ ]:
async def test_strategy(paper_id, market="equities", timeframe="daily", risk_profile="moderate"):
    """Test generating a trading strategy"""
    if not paper_id:
        print("No paper ID provided")
        return None
        
    print(f"Testing strategy endpoint for paper ID: {paper_id}")
    print(f"Market: {market}, Timeframe: {timeframe}, Risk Profile: {risk_profile}")
    start_time = time.time()
    
    async with httpx.AsyncClient(timeout=120.0) as client:  # Longer timeout for strategy generation
        response = await client.post(
            f"{API_BASE_URL}/strategy/",
            json={
                "paper_ids": [paper_id],
                "market": market,
                "timeframe": timeframe,
                "risk_profile": risk_profile,
                "additional_context": "Focus on practical implementation with clear entry/exit rules"
            }
        )
        
        print(f"Status code: {response.status_code}")
        elapsed = time.time() - start_time
        print(f"Response time: {elapsed:.2f} seconds")
        
        if response.status_code == 200:
            strategy = response.json()
            print(f"Strategy name: {strategy.get('strategy_name', 'N/A')}")
            print(f"Code length: {len(strategy.get('python_code', ''))} characters")
            
            # Preview the description
            description = strategy.get('description', '')
            print(f"\nDescription: {description[:200]}...")
            
            return strategy
        else:
            print(f"Error generating strategy: {response.text}")
            return None

In [ ]:
# Run the strategy generation test
strategy = await test_strategy(paper_id)

# Display the generated Python code
if strategy and 'python_code' in strategy:
    print("\nGenerated Python Code:")
    print("```python")
    print(strategy['python_code'])
    print("```")
    
    print("\nUsage Notes:")
    print(strategy.get('usage_notes', 'Not available'))
    
    print("\nLimitations:")
    print(strategy.get('limitations', 'Not available'))

## 7. Testing Streaming Strategy Generation

In [ ]:
async def test_strategy_streaming(paper_id, market="crypto", timeframe="hourly", risk_profile="aggressive"):
    """Test streaming strategy generation"""
    if not paper_id:
        print("No paper ID provided")
        return
        
    print(f"Testing streaming strategy endpoint for paper ID: {paper_id}")
    print(f"Market: {market}, Timeframe: {timeframe}, Risk Profile: {risk_profile}")
    start_time = time.time()
    
    # This is a simplified streaming test - in a real application you would
    # process the streaming response as it comes in
    async with httpx.AsyncClient(timeout=120.0) as client:
        with client.stream(
            "POST",
            f"{API_BASE_URL}/strategy/stream",
            json={
                "paper_ids": [paper_id],
                "market": market,
                "timeframe": timeframe,
                "risk_profile": risk_profile,
                "additional_context": "Focus on high-frequency trading aspects if applicable"
            }
        ) as response:
            print(f"Status code: {response.status_code}")
            if response.status_code == 200:
                print("Receiving streaming response...")
                # Just print the first few chunks to demonstrate streaming
                chunks_received = 0
                async for chunk in response.aiter_text():
                    chunks_received += 1
                    # Print first 3 chunks and then just show progress
                    if chunks_received <= 3:
                        print(f"\nChunk {chunks_received}:\n{chunk}")
                    if chunks_received % 10 == 0:
                        print(f"Received {chunks_received} chunks...")
                        
                elapsed = time.time() - start_time
                print(f"\nStreaming completed in {elapsed:.2f} seconds")
                print(f"Received {chunks_received} total chunks")
            else:
                print(f"Error with streaming: {response.text}")

In [ ]:
# Run the streaming strategy test
# Note: This is optional and may not work if streaming isn't implemented
await test_strategy_streaming(paper_id)

## All Tests Complete

The notebook has finished testing all major endpoints of the ResearchTrader API.